# Novartis - Accumulations
## Beachten
* OCR-CSV muss bereits vorliegen (siehe `Novartis - List`)


In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import CSV

In [3]:
# Read CSV into DataFrame
header = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']  
df = pd.read_csv("Novartis_payment-disclosure-pharma-switzerland-2017-de.csv", sep=";", header=None, names=header)

## Format Table

In [39]:
df_export = df.copy()

#Remove "s"
df_export.loc[df_export['empty'] == 's', 'empty'] = np.NaN

#Shift
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[[0, 2, 3, 4, 5, 12]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Diesen Empfängern zuzuordnender', na=False)]
recipients = df_export[df_export.type.str.contains('Zahl der Empfänger unter', na=False)]
rnd = df_export[-1:]
rnd['total'] = rnd['type']

df_export = pd.concat([amount, recipients, rnd])

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

export_acumulations(df_export, 'novartis')

saved


/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')